In [1]:
import pandas as pd
import numpy as np
import os

# See github.com/hora-search/horapy
try:
    import horapy
except:
    os.system('pip install horapy')
    
from horapy import HNSWIndex

from numpy.random import Generator, PCG64
rng = Generator(PCG64())

In [2]:
# Load the existing IMDB vectorized data
DATA_DIR = '/mnt/512G_hd/data/IMDB/'
text_df = pd.read_parquet(DATA_DIR + 'IMDB_sentiment.parquet') 
    
print(text_df.shape)
text_df.head()

(50000, 3)


,review,sentiment,vector
0,One of the other reviewers has mentioned that ...,positive,"[0.030099334195256233, 0.05041766166687012, -0..."
1,A wonderful little production. <br /><br />The...,positive,"[-0.012201831676065922, 0.05196141451597214, -..."
2,I thought this was a wonderful way to spend ti...,positive,"[0.014258116483688354, -0.0791383683681488, 0...."
3,Basically there's a family where a little boy ...,negative,"[-0.04172040894627571, 0.010464057326316833, -..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[-0.03167528286576271, 0.00642423564568162, -0..."


In [3]:
n_samples, embedding_dim = text_df.shape
sample_ar = np.empty((n_samples, 384), 'float')
for i in range(n_samples):
    x = text_df.values[i,2]
    sample_ar[i] = x[np.newaxis,:]
    
sample_ar.shape

(50000, 384)

In [4]:
sample_ar1 = np.float32(np.random.rand(1000, 50))

n_samples, em_dimension = sample_ar1.shape
n_samples, em_dimension, type(sample_ar1[0,:])

(1000, 50, numpy.ndarray)

In [19]:
# Select one item as a target.

import numpy as np
from horapy import HNSWIndex
# dimension = 50
# n = 1000

# sample_ar1 = np.float32(np.random.rand(n, dimension))
n_samples, em_dimension = sample_ar.shape

embedding_index = HNSWIndex(em_dimension, "usize")

target = rng.choice(n_samples)

for i in range(0, n_samples):
    if i != target:
        # Add an embedding vector to the index
        a_sample = np.float32(sample_ar[i,:])
        # print(a_sample.shape, type(a_sample))
        embedding_index.add(a_sample,i)
    else:
        chosen = np.float32(sample_ar[i,:])
        
embedding_index.build('euclidean')
embedding_index.search(chosen,10)

[23481, 27493, 6581, 25374, 5780, 32159, 10000, 9403, 16256, 41433]

In [9]:
i, type(chosen), target, len(chosen), len(np.float32(sample_ar1[target,:]))

(999, numpy.ndarray, 829, 50, 50)

In [10]:

import numpy as np
from horapy import HNSWIndex

dimension = 50
n = 1000

# init index instance
index = HNSWIndex(dimension, "usize")

samples = np.float32(np.random.rand(n, dimension))
print(f'shape {samples.shape}')
print(type(samples[0,:]))
for i in range(0, len(samples)):
    # add node
    x = np.float32(samples[i])
    # print(type(x), x.shape)
    index.add(x, i)

index.build("euclidean")  # build index

target = np.random.randint(0, n)
print(f'target {target}')
# 410 in Hora ANNIndex <HNSWIndexUsize> (dimension: 50, dtype: usize, max_item: 1000000, n_neigh: 32, n_neigh0: 64, ef_build: 20, ef_search: 500, has_deletion: False)
# has neighbors: [410, 736, 65, 36, 631, 83, 111, 254, 990, 161]
print("\n{} in {} \nhas neighbors: {}".format(
    target, index, index.search(samples[target], 10)))  # search

shape (1000, 50)
<class 'numpy.ndarray'>
target 671

671 in Hora ANNIndex <HNSWIndexUsize> (dimension: 50, dtype: usize, max_item: 1000000, n_neigh: 32, n_neigh0: 64, ef_build: 20, ef_search: 500, has_deletion: False) 
has neighbors: [671, 909, 679, 774, 678, 758, 222, 556, 485, 468]


In [78]:
len(samples)

1000